In [1]:
import sys
sys.path.append('../_amigocloud')

from amigocloud import AmigoCloud
from docx import Document
from docx2pdf import convert
import os
import json

# CONEXIÓN A GOOGLE DRIVE
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
import os

gauth = GoogleAuth()

# Si existe el archivo, intenta cargarlo
if os.path.exists("mycreds.txt"):
    gauth.LoadCredentialsFile("mycreds.txt")

# Si no hay credenciales o son inválidas, forzar nueva autenticación
if gauth.credentials is None or gauth.access_token_expired:
    try:
        gauth.LocalWebserverAuth()
    except:
        print("Error de autenticación. Eliminá 'mycreds.txt' y volvé a intentarlo.")
else:
    gauth.Authorize()

gauth.SaveCredentialsFile("mycreds.txt")
drive = GoogleDrive(gauth)

In [2]:
def get_unidad_local():
    try:
        # ruta del archivo que contiene la información necesaria
        with open('../../unidad_local.txt', 'r') as file:
            unidad = file.readline().strip()
            return unidad
    except FileNotFoundError:
        print("El archivo 'unidad_local.txt' no se encuentra en la ruta especificada.")
    return None

def get_ruta_local():
    try:
        with open('../ruta_local.txt', 'r') as file:
            ruta = file.readline().strip()
            return ruta
    except FileNotFoundError:
        print("El archivo 'ruta_local.txt' no se encuentra en la ruta especificada.")
    return None

In [3]:
def get_ruta_informes():
    try:
        with open('../../ruta_informes.txt', 'r') as file:
            ruta_inf = file.readline().strip()
            return ruta_inf
    except FileNotFoundError:
        print("El archivo 'ruta_local.txt' no se encuentra en la ruta especificada.")
    return None

In [4]:
def cargar_tokens(t_name, ruta_tkn):
    with open(ruta_tkn, 'r') as f:
        tokens = json.load(f)
    return tokens.get(t_name)

In [8]:
# RUTAS
unidad = get_unidad_local()
ruta = get_ruta_local()
ruta_completa = os.path.join(unidad, ruta)
ruta_completa

ruta_informes = get_ruta_informes()
directorio_origen = ruta_informes + r'\_informes'
directorio_destino = ruta_informes + r'\_pdf'

# tkns
tkns = cargar_tokens('tkns_amigocloud', os.path.join(ruta_completa + r'\_keys\tkns.json'))
tkn = tkns['cargar_informes']

amigocloud = AmigoCloud(token=tkn)
amigocloud

In [9]:
# Recorre y convierte a pdf archivos .docx que no empiezan con "_"
def convertir_docx_to_pdf():
    for archivo in os.listdir(directorio_origen):
        if archivo.lower().endswith(".docx") and not archivo.startswith("_"):
            ruta_docx = os.path.join(directorio_origen, archivo)
            ruta_pdf = os.path.join(directorio_destino, archivo.replace(".docx", ".pdf"))
            try:
                convert(ruta_docx, ruta_pdf)
                if os.path.exists(ruta_pdf):
                    print(f"✅ Convertido: {archivo}")
                    os.remove(ruta_docx)
                    print(f"🗑️ Eliminado original: {archivo}")
                else:
                    print(f"❌ Error: PDF no generado para {archivo}")
            except Exception as e:
                print(f"⚠️ Error al convertir {archivo}: {e}")

In [10]:
convertir_docx_to_pdf()

In [15]:
data_siglas = {
    'EMPC': {
        'tipo' : 'Evaluacion de mantenimiento preventivo',
        'id_propject' : 34080,
        'dataset' : 'dataset_341171',
        'drive' : '1GFQMfnFYdy9iDj8h8Mhqh_5ZMKMJeKka', 
        'campo' : 'link_reporte'
    },
    'CDSE': {
        'tipo' : 'Calidad de semilla',
        'id_propject' : 35567,
        'dataset' : 'dataset_365887',
        'drive' : '_CALIDAD DE SEMILLA', 
        'campo' : 'link_informe'
    }
}

In [18]:
def cargar_pdf_to_drive(id_drive, title_file, path_file):
    try:
        # 1. Buscar si ya existe un archivo con el mismo nombre en la misma carpeta
        query = f"'{id_drive}' in parents and title = '{title_file}' and trashed = false"
        file_list = drive.ListFile({'q': query}).GetList()

        if file_list:
            # Si ya existe, usamos el primero encontrado
            file = file_list[0]
            print(f"[INFO] El archivo '{title_file}' ya existe. Se reemplazará.")
        else:
            # Si no existe, creamos uno nuevo
            file = drive.CreateFile({
                "mimeType": "application/pdf",
                "parents": [{"kind": "drive#fileLink", "id": id_drive}]
            })
            file['title'] = title_file
            print(f"[INFO] El archivo '{title_file}' no existe. Se creará uno nuevo.")

        # 2. Reemplazar el contenido
        file.SetContentFile(path_file)
        file.Upload()

        # 3. Dar permisos públicos si es nuevo
        if not file_list:
            file.InsertPermission({
                'type': 'anyone',
                'value': 'anyone',
                'role': 'reader'
            })

        print(f"[OK] Archivo '{title_file}' subido exitosamente.")
        return file['alternateLink']

    except Exception as e:
        print(f"[ERROR] No se pudo subir el archivo '{title_file}': {e}")
        return False

In [11]:
contenido = os.listdir(directorio_destino)
contenido

['10697_EMPC_01-04-2025_PAZ REA JAIME EDUARDO_106.pdf',
 '2816_EMPC_02-04-2025_DONOSO BLANCO ALBERTO_112.pdf',
 '2899_EMPC_02-04-2025_EGUEZ ELHAGE JOSE EDUARDO_110.pdf',
 '2899_EMPC_02-04-2025_EGUEZ ELHAGE JOSE EDUARDO_111.pdf',
 '30035_EMPC_09-04-2025_GUTIERREZ AGUILERA WIDEN GUILLERMO_120.pdf',
 '30035_EMPC_09-04-2025_GUTIERREZ AGUILERA WIDEN GUILLERMO_121.pdf',
 '3421_EMPC_09-04-2025_ESPINOZA HURTADO ARMANDO_119.pdf',
 '446_EMPC_01-04-2025_AGUILERA ANEZ JHONNY_102.pdf',
 '446_EMPC_01-04-2025_AGUILERA ANEZ JHONNY_103.pdf',
 '515_EMPC_01-04-2025_AGUILERA OLGA RIVERO VDA. DE_107.pdf',
 '594_EMPC_07-04-2025_AGROPECUARIA FLAMBOYAN_114.pdf',
 '633_EMPC_01-04-2025_AGROP. VICENTE ROCA GIL SRL_104.pdf',
 '633_EMPC_01-04-2025_AGROP. VICENTE ROCA GIL SRL_105.pdf',
 '75_EMPC_31-03-2025_AGUILERA TARADELLES NELSON MARIANO_100.pdf',
 '75_EMPC_31-03-2025_AGUILERA TARADELLES NELSON MARIANO_101.pdf',
 '765_EMPC_09-04-2025_AGROP. CATALA S.R.L._115.pdf',
 '765_EMPC_09-04-2025_AGROP. CATALA S.R.L._116.p

In [12]:
def cargar_link_to_amigocloud(dataset, campo, url, id_proyecto, id):
    query = {'query': f"UPDATE {dataset} SET {campo} = '{url}' WHERE id = {id}"}
    try:
        res = amigocloud.post(
            f'https://app.amigocloud.com/api/v1/projects/{id_proyecto}/sql',
            query
        )
        return True
    except Exception as e:
        print(f"[EXCEPCIÓN] Error al ejecutar la consulta: {e}")
        return False

In [13]:
file

NameError: name 'file' is not defined

In [19]:
for file in contenido:
    partes = file.split('_')
    
    id = partes[-1].split('.')[0]
    sigla = partes[1]
    title_file = file

    info_sigla = data_siglas[sigla]
    id_drive = info_sigla['drive']
    dataset = info_sigla['dataset']
    campo = info_sigla['campo']
    id_proyecto = info_sigla['id_propject']
    
    path_file = os.path.join(directorio_destino, file)
    url_drive = cargar_pdf_to_drive(id_drive, title_file, path_file)
    if url_drive == False:
        continue
    result_amigocloud = cargar_link_to_amigocloud(dataset, campo, url_drive, id_proyecto, id)
    if result_amigocloud == False:
        continue
    print(url_drive)

[INFO] El archivo '10697_EMPC_01-04-2025_PAZ REA JAIME EDUARDO_106.pdf' ya existe. Se reemplazará.
[OK] Archivo '10697_EMPC_01-04-2025_PAZ REA JAIME EDUARDO_106.pdf' subido exitosamente.
https://drive.google.com/file/d/1aEJsv2DE2wiEORz0hEgI2teZg8xjL5oI/view?usp=drivesdk
[INFO] El archivo '2816_EMPC_02-04-2025_DONOSO BLANCO ALBERTO_112.pdf' ya existe. Se reemplazará.
[OK] Archivo '2816_EMPC_02-04-2025_DONOSO BLANCO ALBERTO_112.pdf' subido exitosamente.
https://drive.google.com/file/d/1rc3JDhZw9i8Pebb05701X05ceOyjip8d/view?usp=drivesdk
[INFO] El archivo '2899_EMPC_02-04-2025_EGUEZ ELHAGE JOSE EDUARDO_110.pdf' ya existe. Se reemplazará.
[OK] Archivo '2899_EMPC_02-04-2025_EGUEZ ELHAGE JOSE EDUARDO_110.pdf' subido exitosamente.
https://drive.google.com/file/d/1DzAQzgKQIE9r2abeT6nGSjWpH0xHd510/view?usp=drivesdk
[INFO] El archivo '2899_EMPC_02-04-2025_EGUEZ ELHAGE JOSE EDUARDO_111.pdf' ya existe. Se reemplazará.
[OK] Archivo '2899_EMPC_02-04-2025_EGUEZ ELHAGE JOSE EDUARDO_111.pdf' subido exit